In [7]:

import os
import numpy as np
import nibabel as nb
import scipy
%matplotlib inline
import meld_classifier.mesh_tools as mt
from scipy import stats as st

In [ ]:
def get_adj_mat(surf):

        all_edges = np.vstack(
            [surf["faces"][:, :2], surf["faces"][:, 1:3], surf["faces"][:, [2, 0]]]
        )
        adj_mat = scipy.sparse.coo_matrix(
            (np.ones(len(all_edges), np.uint8), (all_edges[:, 0], all_edges[:, 1])),
            shape=(len(surf["coords"]), len(surf["coords"])),
        ).tocsr()
        return adj_mat
    
def cluster(mask,adj_mat):
        """cluster predictions and threshold based on min_area_threshold

        Args:
            mask: boolean mask of the per-vertex lesion predictions to cluster"""
        n_comp, labels = scipy.sparse.csgraph.connected_components(adj_mat[mask][:, mask])
        islands = np.zeros(len(mask))
        island_count=0
        # only include islands larger than minimum size.
        for island_index in np.arange(n_comp):
            include_vec = labels == island_index
            size = np.sum(include_vec)

            island_count += 1
            island_mask = mask.copy()
            island_mask[mask] = include_vec
            islands[island_mask] = island_count
        return islands
    
def reindex(clustered,prediction):
    """replace cluster numbers"""
    reindexed_clustered = np.zeros(len(clustered),dtype=int)
    for c in np.unique(clustered)[1:]:
        reindexed_clustered[clustered==c]=st.mode(prediction[clustered==c])[0][0]
    return reindexed_clustered

def correct_interpolation_error(input_mgh,input_surf,output_mgh):
    """correct errors due to interpolation from xhemi back to native"""
    prediction = np.array(mt.load_mgh(input_mgh))
    surf = mt.load_mesh_geometry(input_surf)
    adj = get_adj_mat(surf)
    clustered=cluster(prediction>0,adj)
    reindexed_clustered = reindex(clustered,prediction)
    mt.save_mgh(output_mgh,reindexed_clustered,nb.load(input_mgh))
    return

In [ ]:
subdir = '/home/kw350/rds/rds-kw350-meld/test_meld/output/fs_outputs/MELD_H29_3T_FCD_0079/'
input_mgh=os.path.join(subdir,'surf','rh.prediction.mgh')
input_surf=os.path.join(subdir,'surf','rh.white')
output_mgh=os.path.join(subdir,'surf','rh.prediction_corr.mgh')

In [ ]:
correct_interpolation_error(input_mgh,input_surf,output_mgh)